In [290]:
#Made by George A.saed#
##modules
from PIL import Image, ImageDraw
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:

##remove color and import mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
gray_scale = 255
x_train = x_train.astype('float32') / gray_scale
x_test = x_test.astype('float32') / gray_scale

##test conversion
print("input matrix (x_train)" , x_train.shape )
print("output matrix (y_train)" , y_train.shape )
print("input matrix (x_test)" , x_test.shape )
print("output matrix (y_test)" , y_test.shape )
# Visualizing 10 images from the training data
fig , ax = plt.subplots(2 , 5)
for i in range(2):
 for j in range(5):
     randomp = np.random.randint(0 , 10000)
     ax[i][j].imshow(x_train[randomp].reshape(28 , 28) , aspect ='auto' ,cmap = 'gray' )
     ax[i][j].axis = ('off')
     
plt.suptitle("Sample Images from MNIST Dataset", fontsize=16)
plt.show()     


In [ ]:
##neural network model
model = Sequential([
    ##input layer flattened
    Flatten(input_shape = (28 , 28)),
    
    ##first layer (256)
    Dense(256 , activation = 'relu'),
    
    ##second layer(128)
    Dense(128 , activation = 'relu'),

    ##output layer (10)
    Dense(10 , activation = 'softmax')
])

In [ ]:
##Compiling the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
##Training the model 
history = model.fit(x_train , y_train , epochs = 5,
                    batch_size = 64
                    )

In [ ]:
##Evaluating the model
results = model.evaluate(x_test , y_test , verbose = 0)
plt.figure(figsize=(12, 5))

# Predict the labels for the test images
predictions = model.predict(x_test)

# Get the predicted classes (the index of the highest probability)
predicted_labels = np.argmax(predictions, axis=1)

# Number of images to display
num_images_to_display = 10

# Set up the plot
plt.figure(figsize=(10, 4))

# Display the images, true labels, and predicted labels
for i in range(num_images_to_display):
    random = np.random.randint(0 , 10000)
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[random], cmap="gray")  # Display the image in grayscale
    plt.title(f"Pred: {predicted_labels[random]}")
    plt.axis('off')  # Hide axes for better visualization

# Show the plot
plt.show()


In [300]:
##saving the model 
model.save('mnist_testma.keras')

In [302]:
model = tf.keras.models.load_model('mnist_testma.keras')

In [ ]:
# Canvas size
canvas_size = 20

# Create a Tkinter window
window = tk.Tk()
window.title("Draw a Digit")

# Set canvas dimensions (20x20)
canvas_width = canvas_size * 10  # Scale up for visibility
canvas_height = canvas_size * 10  # Scale up for visibility

# Create a blank canvas
canvas = tk.Canvas(window, width=canvas_width, height=canvas_height, bg="black")
canvas.pack()

# Initialize a PIL image to store the drawn content
image = Image.new("L", (canvas_size, canvas_size), 0)
draw = ImageDraw.Draw(image)

# Function to clear the canvas
def clear_canvas():
    canvas.delete("all")
    draw.rectangle([0, 0, canvas_size, canvas_size], fill=0)  # Black background

# Function to draw on the canvas
def draw_on_canvas(event):
    # Map canvas coordinates to pixel grid coordinates
    x = event.x // 10
    y = event.y // 10
    radius = 1  # A single pixel to be drawn
    canvas.create_rectangle(x * 10, y * 10, (x + 1) * 10, (y + 1) * 10, fill="white", outline="white")
    draw.rectangle([x, y, x + 1, y + 1], fill=255)

# Function to preprocess the image and make a prediction
def predict_digit():
    # Convert the image to 20x20 for MNIST model (already 20x20)
    image_array = np.array(image)

    # Normalize the image
    image_array = image_array / 255.0  # Normalize to [0, 1]

    # Reshape to match the model input shape: (1, 28, 28, 1)
    resized_image = Image.fromarray(image_array * 255).resize((28, 28))
    image_array = np.array(resized_image)
    
    # Reshape to (1, 28, 28, 1)
    image_array = image_array.reshape(1, 28, 28, 1)

    # Make prediction
    prediction = model.predict(image_array)

    # Get the predicted label
    predicted_label = np.argmax(prediction)

    # Display the prediction
    result_label.config(text=f"Predicted Digit: {predicted_label}")

    # Optionally, display the drawn image for verification
    plt.imshow(image_array.reshape(28, 28), cmap='gray')
    plt.title(f"Predicted Digit: {predicted_label}")
    plt.show()

# Button to predict the drawn digit
predict_button = tk.Button(window, text="Predict", command=predict_digit)
predict_button.pack()

# Button to clear the canvas
clear_button = tk.Button(window, text="Clear", command=clear_canvas)
clear_button.pack()

# Label to show the predicted result
result_label = tk.Label(window, text="Predicted Digit: ", font=("Helvetica", 14))
result_label.pack()

# Bind mouse movement to drawing
canvas.bind("<B1-Motion>", draw_on_canvas)

# Start the Tkinter event loop
window.mainloop()